# Parameter efficient fine-tuning





## Оптимизация предобученных моделей
Для уменьшения готовой модели есть два основных подхода: дистиляция и квантизация. 

**Дистиляция** (knowledge distillation) - это обучение меньшей по размеру модели воспроизводить предсказания большей модели. Исходная модель при таком подходе называется учителем, а меньшая модель - учеником. 

Применять дистиляцию к простым языковым моделям нет особого смысла. Но уместна дистиляция для моделей, обученных на инструкциях. Вот например эксперименты по дистиляции GPT в небольшие модели - https://github.com/mbzuai-nlp/LaMini-LM В целом это очень похоже на [Alpaca](https://github.com/tatsu-lab/stanford_alpaca), но датасет здесь намного больше, а сама модель обучается с нуля (в Alpaca дообучается Llama). И также как и с Alpaca, это не очень то легально - по сути это попытка скопировать модель без доступа к обучающим данным, поэтому лицензия на моделях запрещает коммерческое использование. 


**Квантизация** - это уменьшение размера модели за счет уменьшения точности представления чисел. Веса в модели это просто очень много чисел вида 0.23123125, -1.234559 и для хранения каждого такого числа требуется какое-то количество памяти. Интервал допустимых значений и количество знаков после запятой всегда ограничены, но по умолчанию они достаточно большие и, выясняется, что можно достаточно сильно округлить веса, и при этом, практически не потерять в качестве! 

Разумеется, квантизация сложнее, чем просто округление (https://huggingface.co/blog/hf-bitsandbytes-integration). 

## Оптимизация обучения

Согласно вычислениями вот отсюда - https://blog.eleuther.ai/transformer-math/ - при 8bit представлениях можно уместить в колаб и 6.7b модель. Также она используется в оригинальном [туториале](https://colab.research.google.com/drive/1jCkpikz0J2o20FBQmYmAGdiKmJGOMo-o?usp=sharing#scrollTo=cg3fiQOvmI3Q). 

В любом случае, даже с квантизированной моделью полный файн-тюнинг - это все еще очень дорого. Обучение требует в среднем в 4 раза больше памяти, чем инференс, т.е. модель, которую мы загрузили в 8bit и уместили на 1 гпу, обучать мы не сможем.

К счастью, уже появились методы частичного дообучения. Самый популярный на данный момент - LoRA (https://arxiv.org/abs/2106.09685). 

LoRA реализована в отдельной библиотеке PEFT в экосистеме huggingface. Давайте попробуем дообучить большую модель

### Fine-tuning

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

В итоге я использую модель facebook/opt-1.3b, так как 6.7b не помещается в колаб, а 2.7b модель обучается некорректно.

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b", 
    load_in_8bit=True, 
    device_map='auto',max_length=512,
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b", model_max_length=500,
    padding_side="right",
    use_fast=False)

В этой ячейке все веса изначальной модели замораживаются

In [5]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

In [6]:
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [7]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [8]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=32,
    lora_alpha=32, 
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6291456 || all params: 1322049536 || trainable%: 0.47588655558516074


Параметр r отвечает за внутренюю размерность дополнительных матриц. 

In [9]:
import transformers
from datasets import load_dataset

data = load_dataset("databricks/databricks-dolly-15k")
data = data.map(lambda samples: tokenizer(samples['instruction']), batched=True)
data = data.map(lambda samples: tokenizer(samples['context']), batched=True)
data = data.map(lambda samples: tokenizer(samples['response']), batched=True)

trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=250, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=50, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,2.772200
100,2.723800
150,2.688100
200,2.662400
250,2.677900


TrainOutput(global_step=250, training_loss=2.7049048461914063, metrics={'train_runtime': 1347.1991, 'train_samples_per_second': 2.969, 'train_steps_per_second': 0.186, 'total_flos': 5323947713986560.0, 'train_loss': 2.7049048461914063, 'epoch': 0.27})

Сохраним модель (сохранятся только дополнительные веса)

In [11]:
model.save_pretrained('opt_1.3_lora')

Чтобы загрузить обученную модель нужно сначала загрузить базовую модель, а потом применить к ней LoRa веса

In [1]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "opt_1.3_lora"

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-1.3b", 
                                             return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [2]:
def generate(text, tokenizer, model):
  batch = tokenizer(text, return_tensors='pt').to('cuda')
  output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.0, no_repeat_ngram_size=2)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

Давайте попробуем сделать несколько предсказаний, используя базовую модель, чтобы потом сравнить с дообученной

In [3]:
instruction = "Name the most popular language"
generate(instruction,  tokenizer, model)

'Name the most popular language in the world\n\nThe most common language spoken in a country is the one that is most commonly used in that country.\nFor example, in India, Hindi is spoken by more than 80% of the population. In the United States, English'

In [4]:
instruction = "How can I stay healthy?"
generate(instruction,  tokenizer, model)

'How can I stay healthy?\n\nI am a very active person. I love to run, bike, swim, and play tennis. But I am also a mom of two young children and a wife to a wonderful man.\nMy husband and I have been together for almost'

Загрузим LoRA веса и попробуем те же промпты

In [5]:
model = PeftModel.from_pretrained(model, peft_model_id)

In [6]:
instruction = "Name the most popular language"
generate(instruction,  tokenizer, model)

'Name the most popular language in the world:\n\nEnglish\nFrench\nSpanish\nGerman\nItalian\nJapanese\nChinese\nRussian\nArabic\nHindi\nKorean\nThai\nMalay\nTamil\nBengali\nUrdu\nFars'

In [7]:
instruction = "How can I stay healthy?"
generate(instruction,  tokenizer, model)

'How can I stay healthy?\n\nEat healthy.\nDrink water. \nGet enough sleep. Sleep is important. It helps you feel rested and ready to face the day. You can get enough rest by sleeping in a dark room, with a fan on, or'

In [8]:
instruction = "What is the advice to quit smoking"
generate(instruction,  tokenizer, model)

'What is the advice to quit smoking?\n\nQuitting smoking is a difficult task. It is not easy to give up smoking. You have to be patient and persistent. The first step is to stop smoking for a few days. Then you have a break from smoking and then you'

In [9]:
instruction = "What is better: jogging or swimming?"
generate(instruction,  tokenizer, model)

'What is better: jogging or swimming?\n\nJogging is a great way to get your heart rate up and burn calories. It is also a good way for you to burn off excess water weight. However, swimming is more effective at burning calories and is much more enjoyable. Swimming'

Все отлично работает!